In [1]:
from sklearn.neighbors import KNeighborsRegressor as KNN
from sklearn.preprocessing import StandardScaler

from joblib import dump, load
import pandas as pd
import numpy as np

from psyke import Extractor, Clustering, Target
from psyke.extraction.hypercubic.strategy import AdaptiveStrategy
from psyke.extraction.hypercubic import Grid, FeatureRanker
from psyke.utils.logic import pretty_theory

In [2]:
def getTrainTest(data, testB):
    b = bartels[bartels.n==testB]
    t0, t1 = b.t0.values[0], b.t1.values[0]
    idx = (data.index >= t0) & (data.index < t1)
    return data[~idx], data[idx]

def getScaler(train, name):
    scaler = StandardScaler().fit(train)
    dump(scaler, f"scalers/scalerV{name}.joblib")
    normalization = {key: (m, s) for key, m, s in zip(train.columns, scaler.mean_, scaler.scale_)}
    return scaler, pd.DataFrame(scaler.transform(train), columns=train.columns), normalization

In [3]:
def gridrex(model, train, test, normalization):
    ranked = FeatureRanker(train.columns).fit(model, train.iloc[:, :-1]).rankings()
    gridREx = Extractor.gridrex(model, Grid(1, AdaptiveStrategy(ranked, [(0.6, 3)])),
                                threshold=5, min_examples=1, normalization=normalization)
    gridREx.extract(train)
    return gridREx.brute_predict(test), gridREx.n_rules, sum([p is None for p in gridREx.predict(test)])

def cart(model, train, test, normalization):
    CART = Extractor.cart(model, max_depth=5, max_leaves=7, normalization=normalization)
    CART.extract(train)
    return CART.predict(test), CART.n_rules, 0

def cosmik(model, train, test, normalization):
    COSMiK = Extractor.cosmik(model, max_components=10, k=100, patience=10, close_to_center=True,
                              output=Target.REGRESSION, normalization=normalization)
    COSMiK.extract(train)
    return COSMiK.brute_predict(test), COSMiK.n_rules, sum([p is None for p in COSMiK.predict(test)])

def creepy(model, train, test, normalization):
    CReEPy = Extractor.creepy(model, clustering=Clustering.cream, depth=5, error_threshold=5, gauss_components=10,
                              output=Target.REGRESSION, normalization=normalization)
    CReEPy.extract(train)
    return CReEPy.brute_predict(test), CReEPy.n_rules, sum([p is None for p in CReEPy.predict(test)])

In [5]:
bartels = pd.read_csv("data/bartels.csv", parse_dates = [1, 2])

TESTB = [i for i in range(2491, 2509)]

predicted = {'index': [], 'V': [], 'model': [], 'GridREx': [], 'CART': [], 'COSMiK': [], 'CReEPy': []}

rules = {'BR': [], 'GridREx': [], 'CART': [], 'COSMiK': [], 'CReEPy': []}

missed = {'BR': [], 'GridREx': [], 'CART': [], 'COSMiK': [], 'CReEPy': []}

for testB in TESTB:
    rules['BR'].append(testB)
    missed['BR'].append(testB)
    print(testB, end='\r')

    data = pd.read_csv(f'data/halffuzzycoefs2B.csv', parse_dates=[0], index_col=0)
    train, test = getTrainTest(data, testB)

    predicted['index'] += list(test.index.values)
    predicted['V'] += list(test.V.values)

    scaler, scaledTrain, normalization = getScaler(train, f"test{testB}")
    scaledTest = pd.DataFrame(scaler.transform(test), columns=test.columns).iloc[:, :-1]
    m, s = normalization[test.columns[-1]]

    model = KNN(200, weights='distance', p=1).fit(scaledTrain.iloc[:, :-1], scaledTrain.iloc[:, -1])
    #dump(model, f"models/RF/{k}_{name}_{testB}.joblib")
    predicted['model'] += list(model.predict(scaledTest) * s + m)

    for name, fun in zip(['GridREx', 'CART', 'COSMiK', 'CReEPy'], [gridrex, cart, cosmik, creepy]):
        print(testB, name, end='\r')
        #if name in ['GridREx', 'CART', 'COSMiK']:
        #    continue
        pred, n, miss = fun(model, scaledTrain, scaledTest, normalization)
        predicted[name] += list(pred)
        rules[name].append(n)
        missed[name].append(miss)

KeyboardInterrupt: 

In [ ]:
pd.DataFrame(predicted)

In [ ]:
pd.DataFrame(rules)